In [15]:
import torch
import sys
import os
sys.path.insert(0, "..")
from models.LSenTiMent import LSenTiMent
from src.train_test import get_dataloaders
import pickle
import pandas as pd

In [11]:
model = LSenTiMent(
    input_size=1,
    hidden_size=64,
    num_layers=1,
    window_size=10,
    sentiment=True,
    dropout_rate=0.2,
    mlp_hidden_size=64,
    mlp_layers=6
)

state_dict = torch.load("../models/model_no_sentiment.pt", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for LSenTiMent:
	Missing key(s) in state_dict: "mlp.9.weight", "mlp.9.bias", "mlp.12.weight", "mlp.12.bias", "mlp.15.weight", "mlp.15.bias", "mlp.18.weight", "mlp.18.bias", "mlp.21.weight", "mlp.21.bias". 
	size mismatch for mlp.6.weight: copying a param with shape torch.Size([1, 64]) from checkpoint, the shape in current model is torch.Size([64, 64]).
	size mismatch for mlp.6.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([64]).

In [ ]:
data_path = "../data/split"

with open(os.path.join(data_path, "X_train.pkl"), "rb") as f:
        X_train = pickle.load(f)

with open(os.path.join(data_path, "y_train.pkl"), "rb") as f:
    y_train = pickle.load(f)

with open(os.path.join(data_path, "X_val.pkl"), "rb") as f:
    X_val = pickle.load(f)

with open(os.path.join(data_path, "y_val.pkl"), "rb") as f:
    y_val = pickle.load(f)

X_train = torch.cat(X_train)
y_train = torch.cat(y_train)

X_val = torch.cat(X_val)
y_val = torch.cat(y_val)

train_dataloader, val_dataloader = get_dataloaders(X_train, y_train, X_val, y_val, batch_size=64)

In [ ]:
import pandas as pd
import os
import sys
from tqdm import tqdm
sys.path.insert(0, ".")
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '.', 'finBERT')))
import finBERT.finbert.finbert as finbert
from transformers import AutoModelForSequenceClassification
import nltk
import logging

logging.getLogger().setLevel(logging.ERROR)

def get_sentiment(text, model):
    # with suppress_output():
    result = finbert.predict(text, model, use_gpu=True)

    return result["sentiment_score"].mean()

def get_sentiment_for_df(df):
    print("Loading model...")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert",num_labels=3,cache_dir=None)

    tqdm.pandas(desc="Analyzing sentiment")
    df["sentiment"] = df["description"].progress_apply(lambda x: get_sentiment(x, model))

    return df

def main():
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    nltk.download('punkt_tab')

    print("Reading data...")

    df = pd.read_csv("./financial_tweets.csv")
    df = df[["timestamp", "description"]]
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].dt.date

    df = df.dropna(subset=['description'])

    # grouped_df = df.groupby('timestamp')['description'].apply(lambda x: ' '.join(x)).reset_index()

    df = get_sentiment_for_df(df)

    df.to_csv("./financial_tweets_sentiments.csv", index=False)

    # df = get_sentiment_for_df(df)

    # df.to_csv("./financial_tweets_sentiments.csv", index=False)

if __name__ == "__main__":
    main()